In [2]:
import requests
import json
import pymongo
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import gensim
# from bson.objectid import ObjectId

# Connect to db

In [11]:
db_username = 'sveta'
db_password = 'Makhela123'

In [12]:
db = MongoClient("mongodb+srv://" + db_username + ":" + db_password + "@makhela-qvsh8.mongodb.net/Makhela")

In [13]:
mydb = db["Makhela"]

In [14]:
mydb.list_collection_names()

['keywords', 'posts', 'opinion_leaders']

# Opinion Leaders

In [15]:
mycol = mydb["opinion_leaders"].find()

In [16]:
leaders = {}

for leader in mycol:
    try:
        leaders[leader["twitter_id"]] = leader
        leader["posts"] = []
#         print(leader["twitter_id"])
    except:
        continue

In [17]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': True,
 'posts': []}

# Posts

In [18]:
mycol = mydb["posts"].find()

In [19]:
posts = {}

for post in mycol:
    try:
        posts[post["post_id"]] = post
    except:
        continue

In [20]:
posts[1235946337331343366]

{'_id': ObjectId('5e6407df124e74ea22f2a87e'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235946337331343366,
 'full_text': '@michaeltanchum Yep, this applies globally. The need for truth, data and transparency as well.',
 'date_created': datetime.datetime(2020, 3, 6, 15, 12, 24),
 'in_reply_to_status_id': 1235945439180054528,
 'in_reply_to_status_text': '@dassakaye Exactly. Never has made it more clear in the U.S. that good governance is a matter of life and death.',
 'in_reply_to_status_user_id': 'michaeltanchum',
 'quoted_status_id': 'None',
 'quoted_status_text': 'None',
 'quoted_status_user_id': 'None',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None'}

In [21]:
posts[1235944924853317632]

{'_id': ObjectId('5e6407e0124e74ea22f2a87f'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235944924853317632,
 'full_text': 'Not surprising that countries in MENA region with poor governance and multiple domestic and regional crises may not be able to contain #CoronaVirusChallenge and are now focused on mitigation. https://t.co/xCOHRuFpLl',
 'date_created': datetime.datetime(2020, 3, 6, 15, 6, 47),
 'in_reply_to_status_id': None,
 'in_reply_to_status_text': 'None',
 'in_reply_to_status_user_id': 'None',
 'quoted_status_id': 1235931273073483777,
 'quoted_status_text': "LEBANON #Coronavirus Crisis:  #COVID19 spreading in a country that has not been able to buy medical supplies for 5 months. Now Lebanon's health minister says the country is no longer focused on containing the virus https://t.co/NLSSH1PSMT",
 'quoted_status_user_id': 'michaeltanchum',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None'}

In [22]:
def text(dict):
    if dict['retweeted_status_text'] != 'None':
        return dict['retweeted_status_text']
    elif dict['in_reply_to_status_text'] != 'None':
        return dict['in_reply_to_status_text']
    elif dict['quoted_status_text'] != 'None':
        return dict['quoted_status_text']
    else:
        return dict['full_text']

In [23]:
print(text(posts[1235944924853317632]))

LEBANON #Coronavirus Crisis:  #COVID19 spreading in a country that has not been able to buy medical supplies for 5 months. Now Lebanon's health minister says the country is no longer focused on containing the virus https://t.co/NLSSH1PSMT


In [24]:
words_from_posts = []
for key, value in posts.items():
    try:
        post_row = []
        post_row.append(key)
        post_row.append(text(posts[key])) 
#         words_from_posts.append(text(posts[key])) 
        words_from_posts.append(post_row)
    except:
        continue

In [25]:
pr = pd.DataFrame(words_from_posts)
pr

,0,1
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...
2,1235938139119562754,A further isolated and weakened Iran will like...
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri..."
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...
...,...,...
2547,1241082839266967555,@MaghaweirThowra thwarted a #drug smuggling op...
2548,1241082761785626625,This is how #assadregime and #Hezbollah conti...
2549,1241082595435253768,"Maghawir al-Thowra (@MaghaweirThowra), a @CJTF..."
2550,1241036286334578695,This is essential reading on why drastic suppr...


In [26]:
pr[1]

0       @dassakaye Exactly. Never has made it more cle...
1       LEBANON #Coronavirus Crisis:  #COVID19 spreadi...
2       A further isolated and weakened Iran will like...
3       Coronavirus: Iran temporarily frees 54,000 pri...
4       @dassakaye Not necessarily to be honest. #Saud...
                              ...                        
2547    @MaghaweirThowra thwarted a #drug smuggling op...
2548    This is how #assadregime  and #Hezbollah conti...
2549    Maghawir al-Thowra (@MaghaweirThowra), a @CJTF...
2550    This is essential reading on why drastic suppr...
2551    Looking back, says @tylercowen, historians may...
Name: 1, Length: 2552, dtype: object

In [27]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(pr[1])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [28]:
df.head()

,000,00am,00q3870mh1,01,0120kg,03,035mi,03am,075,083,...,全く同感なのだが,同時にトランプ政権はきっとこの議論を理解しないだろうとも確信している,新型コロナウィルスは米国の対イラン政策の転換のチャンスという記事とその紹介連ツイ,欧州が同航空機の空域入域を拒否したからとのこと,동해상으로,미상,발사,발사체,북한,합참
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.183098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
len(posts)

2552

In [83]:
df.iloc[0,:].max()

0.36977650402877305

In [84]:
words = []
for i in range(len(df)):
    words.append(df.iloc[i,:][df.iloc[i,:]==df.iloc[i,:].max()].index.values.tolist())

In [85]:
len(words)

2552

In [86]:
pr['words'] = words

In [87]:
pr

,0,1,words
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...,[dassakaye]
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...,[lebanon]
2,1235938139119562754,A further isolated and weakened Iran will like...,"[emboldening, isolated, pscktckw8m]"
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri...","[ixktbh9qen, unconditional]"
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...,"[managem, necessarily, ph93rk6s04]"
...,...,...,...
2547,1241082839266967555,@MaghaweirThowra thwarted a #drug smuggling op...,[smuggling]
2548,1241082761785626625,This is how #assadregime and #Hezbollah conti...,"[assadregime, vrsgfkolz4, warcrimes]"
2549,1241082595435253768,"Maghawir al-Thowra (@MaghaweirThowra), a @CJTF...","[bound, cjtfoir, eix12wazrx, garrison, interdi..."
2550,1241036286334578695,This is essential reading on why drastic suppr...,"[perilously, r9e7bympnv, sane]"


In [88]:
for i in range(len(pr)):
    try:
        posts[pr.loc[i,0]]['words'] = pr.loc[i,'words']
    except:
        continue


In [89]:
posts

{1235946337331343366: {'_id': ObjectId('5e6407df124e74ea22f2a87e'),
  'leader_twitter_id': 2696666707,
  'post_id': 1235946337331343366,
  'full_text': '@michaeltanchum Yep, this applies globally. The need for truth, data and transparency as well.',
  'date_created': datetime.datetime(2020, 3, 6, 15, 12, 24),
  'in_reply_to_status_id': 1235945439180054528,
  'in_reply_to_status_text': '@dassakaye Exactly. Never has made it more clear in the U.S. that good governance is a matter of life and death.',
  'in_reply_to_status_user_id': 'michaeltanchum',
  'quoted_status_id': 'None',
  'quoted_status_text': 'None',
  'quoted_status_user_id': 'None',
  'retweeted_status_id': 'None',
  'retweeted_status_text': 'None',
  'retweeted_status_user_id': 'None',
  'words': ['dassakaye']},
 1235944924853317632: {'_id': ObjectId('5e6407e0124e74ea22f2a87f'),
  'leader_twitter_id': 2696666707,
  'post_id': 1235944924853317632,
  'full_text': 'Not surprising that countries in MENA region with poor governan

# Add posts to opinion leader

In [90]:
for key, value in posts.items():
    try:
        leaders[value['leader_twitter_id']]["posts"].append(key)
    except:
        continue

In [91]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': True,
 'posts': [1235946337331343366,
  1235944924853317632,
  1235938139119562

In [92]:
#retweeted_status_text | in_reply_to_status_text | quoted_status_text ==> full_text

In [93]:
# words_from_posts = []
# for post in posts:
#     words_from_posts.append(post["full_text"])

In [94]:
# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(words_from_posts)
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# df = pd.DataFrame(denselist, columns=feature_names)

In [95]:
# for i in range(len(df)):
#     posts[i]["words"] = df.iloc[i,:][df.iloc[i,:]==df.iloc[i,:].max()].index.values.tolist()

# LDA

In [96]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [97]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [98]:
import spacy


In [99]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [100]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [101]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [102]:
def LDA(posts_corpus):
    df = pd.DataFrame(posts_corpus) 
    df = df.drop_duplicates()
    df = df.rename({0: 'leader', 1: 'post'}, axis='columns')
    data = df.post.values.tolist()
    data = [re.sub('\s+', ' ', sent) for sent in data]
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])


    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=5, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    return lda_model

In [111]:
posts_corpus = []

for post in leaders[2696666707]['posts']:
    

[1235946337331343366,
 1235944924853317632,
 1235938139119562754,
 1235161759423614977,
 1234901226807939072,
 1234900896493912064,
 1234899655038623744,
 1234898234654375936,
 1234498947554500608,
 1234190806795218944,
 1233968148367364097,
 1233825820759576577,
 1233544954120744962,
 1233534449293373446,
 1233460930882756608,
 1233147244364189696,
 1233146036111036417,
 1233144044370313216,
 1233089606196662272,
 1233052100264775680,
 1236394102721146880,
 1236412118720638976,
 1236411577026277376,
 1236418283923881986,
 1237165112080740352,
 1237163704057131008,
 1237162731435741185,
 1237134870477864960,
 1237074454334689282,
 1237074350596952064,
 1237035744297766912,
 1238167053204254721,
 1238119172128657408,
 1237943154155741185,
 1237939517572517888,
 1237938601561739265,
 1237937792254013440,
 1237916434069512192,
 1237915476493103104,
 1237914667625791490,
 1237909020138041344,
 1237885694921469952,
 1237880668949901313,
 1237879557786484738,
 1237874133553139714,
 123787155

In [109]:
for key, value in leaders.items():
    posts_corpus = []
    try:   
        for post in value['posts']:
            leader_posts = []
            leader_posts.append(key)
            leader_posts.append(text(posts[post]))
            posts_corpus.append(leader_posts)
        leaders[key]['topics'] = LDA(posts_corpus).print_topics()
        print(LDA(posts_corpus).print_topics())
    except:
        print(key)
        continue

NameError: name 'e' is not defined

In [106]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': True,
 'posts': [1235946337331343366,
  1235944924853317632,
  1235938139119562

In [45]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [46]:

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# Neo4j

In [300]:
from neo4j.v1 import GraphDatabase
import nxneo4j

In [301]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "pw"))

In [348]:
config = {
    "node_label": "Leader",
    "relationship_type": "FOLLOWS",
    "identifier_property": "leaders"
}

In [346]:
G = nxneo4j.DiGraph(driver ,config)

In [ ]:
CREATE (Carrie:Person {name:'Carrie-Anne Moss', born:1967})

# Graph

In [47]:
import networkx as nx

In [48]:
G = nx.DiGraph()

In [49]:
for key, value in leaders.items():
    G.add_node(key)
    try:
        G.nodes[key]['twitter_id'] = value['twitter_id']
    except:
        pass
    try:
        G.nodes[key]['full_name'] = value['full_name']
    except:
        pass
    try:
        G.nodes[key]['twitter_screen_name'] = value['twitter_screen_name']
    except:
        pass

    try:
        G.nodes[key]['twitter_description'] = value['twitter_description']
    except:
        pass
    try:
        G.nodes[key]['twitter_followers_count'] = value['twitter_followers_count']
    except:
        pass
    try:
        G.nodes[key]['twitter_friends_count'] = value['twitter_friends_count']
    except:
        pass
    try:
        G.nodes[key]['twitter_location'] = value['twitter_location']
    except:
        pass
    try:
        G.nodes[key]['new_leader'] = value['new_leader']
    except:
        pass
    try:
        G.nodes[key]['twitter_statuses_count'] = value['twitter_statuses_count']
    except:
        pass
    try:
        G.nodes[key]['level_of_certainty'] = value['level_of_certainty']
    except:
        pass
    try:
        G.nodes[key]['twitter_profile_image'] = value['twitter_profile_image']
    except:
        pass
    try:
        G.nodes[key]['community_following'] = value['community_following']
    except:
        pass
    try:
        for follow in value['community_following']:
            G.add_edge(key, follow)
    except:
        continue
#     G.add_edge(2696666707, 22803302)

In [50]:
# len(G)

In [51]:
# G.nodes

In [52]:
# list(G.nodes(data=True))

In [53]:
# list(G.edges())

In [54]:
# import matplotlib.pyplot as plt

In [55]:
# Gr = nx.cycle_graph(G)
# pos = nx.circular_layout(Gr)

# # nx.draw_networkx_nodes(G, pos, node_size=150, node_color='blue', alpha=0.3)
# # nx.draw_networkx_edges(G, pos, width=2, alpha=0.3, edge_color='green')
# # nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')
# # # # default
# # plt.figure(1)
# # smaller nodes and fonts
# # plt.figure(2)
# # larger figure size
# # nx.draw(G,pos,node_size=60,font_size=8) 
# plt.figure(figsize=(12,12)) 
# nx.draw(G,pos,node_size=60)
# plt.show()
# # # plt.savefig('data.png')  

In [56]:
# print(nx.is_strongly_connected(G))

In [57]:
# isolated = nx.isolates(G)

In [58]:
# for i in isolated:
#     G.remove_node(i)

In [59]:
# nx.center(G)

In [60]:
# nx.periphery(G)

In [61]:
# degree_centrality = nx.degree_centrality(G)

In [62]:
# degree_centrality

In [63]:
# centrality_values = []
# for key, value in degree_centrality.items():
#     centrality_values.append([key, value])

In [64]:
# centrality_values

In [65]:
# centrality_values.sort(key=lambda x: x[1], reverse=True)

In [66]:
# centrality_values

In [67]:
# for leader in centrality_values[:5]:
#     print(leaders[leader[0]])

In [68]:
# for leader in centrality_values[-5:]:
#     print(leaders[leader[0]])

In [69]:
G = nx.DiGraph()

In [70]:
posts[1235946337331343366]

{'_id': ObjectId('5e6407df124e74ea22f2a87e'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235946337331343366,
 'full_text': '@michaeltanchum Yep, this applies globally. The need for truth, data and transparency as well.',
 'date_created': datetime.datetime(2020, 3, 6, 15, 12, 24),
 'in_reply_to_status_id': 1235945439180054528,
 'in_reply_to_status_text': '@dassakaye Exactly. Never has made it more clear in the U.S. that good governance is a matter of life and death.',
 'in_reply_to_status_user_id': 'michaeltanchum',
 'quoted_status_id': 'None',
 'quoted_status_text': 'None',
 'quoted_status_user_id': 'None',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None'}

In [71]:
for key, value in posts.items():
    G.add_node(key)
    try:
        G.nodes[key]['leader_twitter_id'] = value['leader_twitter_id']
    except:
        pass
    try:
        G.nodes[key]['post_id'] = value['post_id']
    except:
        pass
    try:
        G.nodes[key]['full_text'] = value['full_text']
    except:
        pass
    try:
        G.nodes[key]['date_created'] = value['date_created']
    except:
        pass
    try:
        G.nodes[key]['in_reply_to_status_id'] = value['in_reply_to_status_id']
    except:
        pass
    try:
        G.nodes[key]['in_reply_to_status_user_id'] = value['in_reply_to_status_user_id']
    except:
        pass
    try:
        G.nodes[key]['quoted_status_id'] = value['quoted_status_id']
    except:
        pass
    try:
        G.nodes[key]['quoted_status_text'] = value['quoted_status_text']
    except:
        pass
    try:
        G.nodes[key]['quoted_status_user_id'] = value['quoted_status_user_id']
    except:
        pass
    try:
        G.nodes[key]['retweeted_status_id'] = value['retweeted_status_id']
    except:
        pass
    try:
        G.nodes[key]['retweeted_status_text'] = value['retweeted_status_text']
    except:
        pass
    try:
        G.nodes[key]['community_following'] = value['retweeted_status_user_id']
    except:
        pass
    try:
        G.nodes[key]['words'] = value['words']
    except:
        pass
#     try:
#         for follow in value['community_following']:
#             G.add_edge(key, follow)
#     except:
#         continue
#     G.add_edge(2696666707, 22803302)

In [72]:
len(G)

2552

In [73]:
G.nodes

NodeView((1235946337331343366, 1235944924853317632, 1235938139119562754, 1235161759423614977, 1234901226807939072, 1234900896493912064, 1234899655038623744, 1234898234654375936, 1234498947554500608, 1234190806795218944, 1233968148367364097, 1233825820759576577, 1233544954120744962, 1233534449293373446, 1233460930882756608, 1233147244364189696, 1233146036111036417, 1233144044370313216, 1233089606196662272, 1233052100264775680, 1236295343647731712, 1236074467727675397, 1236054077492154369, 1236047440555511809, 1236043011932459009, 1236041867923517441, 1236037620209201155, 1236037619366203401, 1236037618481221635, 1236037617680027649, 1236037616841129986, 1236037615897505794, 1236037614974709762, 1236037614177865728, 1236037613309571073, 1236037612512608257, 1236037611606675456, 1236019308507148290, 1236016962926448640, 1235784107289960449, 1051112878990876675, 1034811088083148801, 846886605445107714, 655695091814801408, 606262100831412224, 1236005993852567553, 1234989287784869889, 123497

In [74]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': True,
 'posts': []}